# FAQ Content Change Detection - Usage Guide

This notebook demonstrates how to use the detection module to identify content changes that require FAQ regeneration.

## Overview

The detection module implements a **checksum-centric approach** with **similarity matching** to detect:
- **NEW content**: Completely new content added
- **MODIFIED content**: Existing content that changed (detected via similarity matching)
- **DELETED content**: Content that was removed
- **UNCHANGED content**: Content with identical checksums

## Key Modules

1. **ChecksumExtractor**: Computes SHA-256 checksums for content tracking
2. **ChecksumFileExtractor**: Extracts checksums from various sources (markdown, CSV, Spark DataFrames)
3. **ContentChangeDetector**: Detects changes using pure checksum comparison + similarity matching
4. **DatabaseQueries**: Retrieves previous checksums and stores results

## Use Case Context

**Scenario**: You have an HR policy document that gets updated periodically. You need to:
1. Extract content checksums from the NEW version
2. Retrieve checksums from the PREVIOUS version (from database)
3. Detect what changed (new, modified, deleted content)
4. Store change detection results
5. Use these results to selectively regenerate FAQs

---

## Setup and Imports

In [1]:
import sys
from pathlib import Path
from datetime import datetime
import uuid

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent.parent))

# Import detection modules
from granular_impact.detection.checksum_extractor import ChecksumExtractor
from granular_impact.detection.checksum_file_extractor import ChecksumFileExtractor
from granular_impact.detection.content_change_detector import ContentChangeDetector
from granular_impact.detection.database_queries import ChangeDetectionQueries

# Import models
from granular_impact.database.models import ChangeType, ContentChange

print("✅ Imports successful!")

✅ Imports successful!


---

## Part 1: Basic Checksum Extraction

### Understanding Checksums

A checksum is a **unique fingerprint** of content. Even a single character change produces a completely different checksum.

This is the foundation of our change detection system.

In [2]:
# Initialize the checksum extractor (default: SHA-256)
checksum_extractor = ChecksumExtractor(algorithm="sha256")

# Example: Compute checksums for sample content
original_content = "Employees are entitled to 10 sick days per year."
modified_content = "Employees are entitled to 12 sick days per year."
identical_content = "Employees are entitled to 10 sick days per year."

checksum_original = checksum_extractor.compute_checksum(original_content)
checksum_modified = checksum_extractor.compute_checksum(modified_content)
checksum_identical = checksum_extractor.compute_checksum(identical_content)

print("🔍 Checksum Analysis:")
print(f"Original:  {checksum_original[:16]}... (64 chars)")
print(f"Modified:  {checksum_modified[:16]}... (64 chars)")
print(f"Identical: {checksum_identical[:16]}... (64 chars)")
print()
print(f"Original == Identical: {checksum_original == checksum_identical}")
print(f"Original == Modified:  {checksum_original == checksum_modified}")
print()
print("💡 Key Insight: Even small changes (10→12) produce completely different checksums!")

🔍 Checksum Analysis:
Original:  944ce32eb97e9b0e... (64 chars)
Modified:  b149802a5bc36e3c... (64 chars)
Identical: 944ce32eb97e9b0e... (64 chars)

Original == Identical: True
Original == Modified:  False

💡 Key Insight: Even small changes (10→12) produce completely different checksums!


---

## Part 2: Extracting Checksums from Files

In production, content comes from various sources:
- **Markdown files** (extracted from PDFs)
- **CSV files** (structured data)
- **Spark DataFrames** (Databricks environment)

Let's simulate extracting checksums from markdown files.

In [3]:
# Initialize file extractor
file_extractor = ChecksumFileExtractor(checksum_algorithm="sha256")

# Create sample markdown files (simulating extracted content)
sample_dir = Path("./sample_content")
sample_dir.mkdir(exist_ok=True)

# Simulate: HR Handbook content across multiple pages
sample_content = {
    "hr_handbook_page_1.md": "# Time Off Policy\n\nEmployees are entitled to 10 sick days per year.",
    "hr_handbook_page_2.md": "# Benefits\n\nFull-time employees receive health insurance coverage.",
    "hr_handbook_page_3.md": "# Remote Work\n\nEmployees may work remotely up to 2 days per week.",
}

# Write sample files
for filename, content in sample_content.items():
    with open(sample_dir / filename, "w", encoding="utf-8") as f:
        f.write(content)

print(f"✅ Created {len(sample_content)} sample markdown files in {sample_dir}/")

# Extract checksums from markdown files
checksums_data = file_extractor.extract_from_markdown_files(
    markdown_dir=sample_dir,
    file_name_pattern="*.md"
)

print(f"\n📊 Extracted {len(checksums_data)} unique checksums:")
for checksum, data in checksums_data.items():
    print(f"  {checksum[:16]}... → Page {data['page_num']} ({data['file_name']})")

print("\n🗂️ Data structure:")
print(f"Type: {type(checksums_data)}")
print(f"Keys: {list(checksums_data.keys())[:1]}")
sample_checksum = list(checksums_data.keys())[0]
print(f"\nSample entry: {checksums_data[sample_checksum]}")

2025-10-24 01:58:04.457 | INFO     | granular_impact.detection.checksum_file_extractor:__init__:39 - ChecksumFileExtractor initialized with sha256
2025-10-24 01:58:04.460 | INFO     | granular_impact.detection.checksum_file_extractor:extract_from_markdown_files:65 - 📂 Extracting checksums from 3 markdown files in sample_content
2025-10-24 01:58:04.461 | DEBUG    | granular_impact.detection.checksum_file_extractor:extract_from_markdown_files:86 -    ✓ hr_handbook_page_1.md → 86248316... (page 1)
2025-10-24 01:58:04.463 | DEBUG    | granular_impact.detection.checksum_file_extractor:extract_from_markdown_files:86 -    ✓ hr_handbook_page_2.md → c51f78cb... (page 2)
2025-10-24 01:58:04.464 | DEBUG    | granular_impact.detection.checksum_file_extractor:extract_from_markdown_files:86 -    ✓ hr_handbook_page_3.md → 3399ce4d... (page 3)
2025-10-24 01:58:04.466 | INFO     | granular_impact.detection.checksum_file_extractor:extract_from_markdown_files:91 - ✅ Extracted 3 unique checksums


✅ Created 3 sample markdown files in sample_content/

📊 Extracted 3 unique checksums:
  86248316e8b0753c... → Page 1 (hr_handbook_page_1.md)
  c51f78cb3f611c72... → Page 2 (hr_handbook_page_2.md)
  3399ce4d456d4f73... → Page 3 (hr_handbook_page_3.md)

🗂️ Data structure:
Type: <class 'dict'>
Keys: ['86248316e8b0753cb7f837920eb4072e3ca84ffc092226bb8d0ea5d2d7956374']

Sample entry: {'text': '# Time Off Policy\n\nEmployees are entitled to 10 sick days per year.', 'markdown_path': 'sample_content\\hr_handbook_page_1.md', 'file_name': 'hr_handbook_page_1.md', 'page_num': 1}


### Extracting from CSV Files

CSV files are useful for structured content data from previous versions stored in volumes.

In [4]:
import csv

# Create a sample CSV file (simulating previous version stored in database/volume)
csv_path = sample_dir / "previous_content.csv"

with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["file_name", "page_number", "content_text"])
    writer.writeheader()
    writer.writerow({
        "file_name": "hr_handbook.pdf",
        "page_number": 1,
        "content_text": "# Time Off Policy\n\nEmployees are entitled to 10 sick days per year."
    })
    writer.writerow({
        "file_name": "hr_handbook.pdf",
        "page_number": 2,
        "content_text": "# Benefits\n\nFull-time employees receive health insurance coverage."
    })

print(f"✅ Created sample CSV: {csv_path}")

# Extract checksums from CSV
csv_checksums_data = file_extractor.extract_from_csv(csv_path)

print(f"\n📊 Extracted {len(csv_checksums_data)} checksums from CSV:")
for checksum, data in csv_checksums_data.items():
    print(f"  {checksum[:16]}... → Page {data['page_number']} (row {data['row_num']})")

2025-10-24 01:58:05.910 | INFO     | granular_impact.detection.checksum_file_extractor:extract_from_csv:118 - 📄 Extracting checksums from CSV: sample_content\previous_content.csv
2025-10-24 01:58:05.911 | INFO     | granular_impact.detection.checksum_file_extractor:extract_from_csv:155 - ✅ Extracted 2 unique checksums from CSV


✅ Created sample CSV: sample_content\previous_content.csv

📊 Extracted 2 checksums from CSV:
  86248316e8b0753c... → Page 1 (row 2)
  c51f78cb3f611c72... → Page 2 (row 3)


---

## Part 3: Content Change Detection

This is the **core** of our system. We compare:
- **Current checksums** (new version)
- **Previous checksums** (old version)

And detect:
1. **Set difference** → New/Deleted checksums
2. **Similarity matching** → Modified content (e.g., "10 days" → "12 days")

### Algorithm Overview

```
Current Checksums:  {A, B, C, D}
Previous Checksums: {A, B, E, F}

Unchanged: A, B (exist in both)
New:       C, D (in current, not in previous)
Deleted:   E, F (in previous, not in current)

Similarity Matching:
- Compare C vs E, C vs F
- Compare D vs E, D vs F
- If similarity >= 0.8 → MODIFIED
- Otherwise → truly NEW/DELETED
```

In [5]:
# Initialize the content change detector
change_detector = ContentChangeDetector(
    checksum_algorithm="sha256",
    similarity_threshold=0.8  # 80% similarity = MODIFIED
)

print("✅ ContentChangeDetector initialized")
print(f"   Algorithm: {change_detector.checksum_extractor.algorithm}")
print(f"   Similarity threshold: {change_detector.similarity_threshold}")
print(f"   Similarity calculator: {change_detector.similarity_calculator.get_algorithm_name()}")
print(f"   Weights: {change_detector.similarity_calculator.weights}")

2025-10-24 01:58:06.838 | INFO     | granular_impact.detection.content_change_detector:__init__:83 - ContentChangeDetector initialized: algorithm=sha256, similarity_threshold=0.8, compute_llm_diffs=True, similarity_weights={'difflib': 1.0}


✅ ContentChangeDetector initialized
   Algorithm: sha256
   Similarity threshold: 0.8
   Similarity calculator: hybrid
   Weights: {'difflib': 1.0}


### Scenario: Updated HR Handbook

Let's simulate a real-world scenario:
- **Previous version**: 10 sick days, health insurance, remote work (2 days/week)
- **New version**: 12 sick days (MODIFIED), health insurance (UNCHANGED), remote work (3 days/week - MODIFIED), + new parental leave policy (NEW)
- One section was removed (DELETED)

In [6]:
# PREVIOUS VERSION (what was in the database)
previous_checksums_data = {
    # Page 1: Time Off - WILL BE MODIFIED
    checksum_extractor.compute_checksum(
        "# Time Off Policy\n\nEmployees are entitled to 10 sick days per year."
    ): {
        "content_text": "# Time Off Policy\n\nEmployees are entitled to 10 sick days per year.",
        "file_name": "hr_handbook.pdf",
        "page_number": 1,
    },
    # Page 2: Benefits - WILL BE UNCHANGED
    checksum_extractor.compute_checksum(
        "# Benefits\n\nFull-time employees receive health insurance coverage."
    ): {
        "content_text": "# Benefits\n\nFull-time employees receive health insurance coverage.",
        "file_name": "hr_handbook.pdf",
        "page_number": 2,
    },
    # Page 3: Remote Work - WILL BE MODIFIED
    checksum_extractor.compute_checksum(
        "# Remote Work\n\nEmployees may work remotely up to 2 days per week."
    ): {
        "content_text": "# Remote Work\n\nEmployees may work remotely up to 2 days per week.",
        "file_name": "hr_handbook.pdf",
        "page_number": 3,
    },
    # Page 4: Old vacation policy - WILL BE DELETED
    checksum_extractor.compute_checksum(
        "# Vacation\n\nEmployees must request vacation 2 weeks in advance."
    ): {
        "content_text": "# Vacation\n\nEmployees must request vacation 2 weeks in advance.",
        "file_name": "hr_handbook.pdf",
        "page_number": 4,
    },
}

# CURRENT VERSION (new extracted content)
current_checksums_data = {
    # Page 1: Time Off - MODIFIED (10 → 12 days)
    checksum_extractor.compute_checksum(
        "# Time Off Policy\n\nEmployees are entitled to 12 sick days per year."
    ): {
        "text": "# Time Off Policy\n\nEmployees are entitled to 12 sick days per year.",
        "file_name": "hr_handbook.pdf",
        "page_num": 1,
        "markdown_path": "./sample_content/hr_handbook_page_1.md",
    },
    # Page 2: Benefits - UNCHANGED (exact same content)
    checksum_extractor.compute_checksum(
        "# Benefits\n\nFull-time employees receive health insurance coverage."
    ): {
        "text": "# Benefits\n\nFull-time employees receive health insurance coverage.",
        "file_name": "hr_handbook.pdf",
        "page_num": 2,
        "markdown_path": "./sample_content/hr_handbook_page_2.md",
    },
    # Page 3: Remote Work - MODIFIED (2 → 3 days)
    checksum_extractor.compute_checksum(
        "# Remote Work\n\nEmployees may work remotely up to 3 days per week."
    ): {
        "text": "# Remote Work\n\nEmployees may work remotely up to 3 days per week.",
        "file_name": "hr_handbook.pdf",
        "page_num": 3,
        "markdown_path": "./sample_content/hr_handbook_page_3.md",
    },
    # Page 5: NEW - Parental Leave (completely new section)
    checksum_extractor.compute_checksum(
        "# Parental Leave\n\nEmployees receive 12 weeks of paid parental leave."
    ): {
        "text": "# Parental Leave\n\nEmployees receive 12 weeks of paid parental leave.",
        "file_name": "hr_handbook.pdf",
        "page_num": 5,
        "markdown_path": "./sample_content/hr_handbook_page_5.md",
    },
}

print("📊 Setup complete:")
print(f"  Previous version: {len(previous_checksums_data)} pages")
print(f"  Current version:  {len(current_checksums_data)} pages")

📊 Setup complete:
  Previous version: 4 pages
  Current version:  4 pages


In [7]:
# Run change detection!
detection_run_id = f"run_{uuid.uuid4().hex[:8]}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
file_name = "hr_handbook.pdf"

print(f"🔍 Running change detection...")
print(f"   File: {file_name}")
print(f"   Run ID: {detection_run_id}")
print()

changes = change_detector.detect_changes(
    file_name=file_name,
    current_checksums_data=current_checksums_data,
    previous_checksums_data=previous_checksums_data,
    detection_run_id=detection_run_id,
)

print(f"\n✅ Detection complete! Found {len(changes)} changes")

2025-10-24 01:58:08.393 | INFO     | granular_impact.detection.content_change_detector:detect_changes:121 - 🔍 Detecting changes for: hr_handbook.pdf
2025-10-24 01:58:08.395 | INFO     | granular_impact.detection.content_change_detector:detect_changes:122 -    Detection run: run_b0f56fa4_20251024_015808
2025-10-24 01:58:08.396 | INFO     | granular_impact.detection.content_change_detector:detect_changes:133 - 📊 Checksum analysis for hr_handbook.pdf:
2025-10-24 01:58:08.397 | INFO     | granular_impact.detection.content_change_detector:detect_changes:134 -    Current:   4 checksums
2025-10-24 01:58:08.398 | INFO     | granular_impact.detection.content_change_detector:detect_changes:135 -    Previous:  4 checksums
2025-10-24 01:58:08.399 | INFO     | granular_impact.detection.content_change_detector:detect_changes:136 -    New:       3 checksums
2025-10-24 01:58:08.400 | INFO     | granular_impact.detection.content_change_detector:detect_changes:137 -    Deleted:   3 checksums
2025-10-24 

🔍 Running change detection...
   File: hr_handbook.pdf
   Run ID: run_b0f56fa4_20251024_015808


✅ Detection complete! Found 5 changes


### Analyze the Results

In [12]:
modified_changes = [c for c in changes if c.change_type == ChangeType.MODIFIED_CONTENT]
modified_changes

[ContentChange(content_checksum='fb38a20a36d536ca85f7c92eab11bfa5a67d38e66c643af627fa6b0562897532', old_checksum='86248316e8b0753cb7f837920eb4072e3ca84ffc092226bb8d0ea5d2d7956374', new_checksum='fb38a20a36d536ca85f7c92eab11bfa5a67d38e66c643af627fa6b0562897532', change_type=<ChangeType.MODIFIED_CONTENT: 'modified_content'>, detected_at=datetime.datetime(2025, 10, 24, 1, 58, 8, 404347), file_name='hr_handbook.pdf', page_number=1, old_content='# Time Off Policy\n\nEmployees are entitled to 10 sick days per year.', new_content='# Time Off Policy\n\nEmployees are entitled to 12 sick days per year.', similarity_score=0.9848484848484849, llm_friendly_diff='Found 1 change(s):\n\nCHANGE #1:\nBEFORE: "# Time Off Policy\n\nEmployees are entitled to 10 sick days per year."\nAFTER:  "# Time Off Policy\n\nEmployees are entitled to 12 sick days per year."\nWHAT CHANGED:\n  "10" → "12"'),
 ContentChange(content_checksum='1b772f4b1a403b0d94f502743942e11487621034ea05ea996bd020713dc40e2b', old_checksum='

In [8]:
# Categorize changes by type
new_changes = [c for c in changes if c.change_type == ChangeType.NEW_CONTENT]
modified_changes = [c for c in changes if c.change_type == ChangeType.MODIFIED_CONTENT]
deleted_changes = [c for c in changes if c.change_type == ChangeType.DELETED_CONTENT]
unchanged_changes = [c for c in changes if c.change_type == ChangeType.UNCHANGED_CONTENT]

print("📊 Change Summary:")
print(f"  NEW:       {len(new_changes)} (requires FAQ generation)")
print(f"  MODIFIED:  {len(modified_changes)} (requires FAQ regeneration)")
print(f"  DELETED:   {len(deleted_changes)} (requires FAQ invalidation)")
print(f"  UNCHANGED: {len(unchanged_changes)} (no action needed)")
print()

# Detail each change type
if modified_changes:
    print("\n🔄 MODIFIED Content (Similarity Matching Detected These):")
    for change in modified_changes:
        print(f"\n  Page {change.page_number}:")
        print(f"    Old checksum: {change.old_checksum[:16]}...")
        print(f"    New checksum: {change.new_checksum[:16]}...")
        print(f"    Similarity:   {change.similarity_score:.3f} (>= 0.8 threshold)")
        print(f"    Old content:  {change.old_content[:60]}...")
        print(f"    New content:  {change.new_content[:60]}...")
        
        # Show LLM-friendly diff if available (use getattr for backward compatibility)
        llm_diff = getattr(change, 'llm_friendly_diff', None)
        if llm_diff:
            print(f"\n    📝 LLM-Friendly Diff:")
            print("    " + "-" * 72)
            for line in llm_diff.split("\n"):
                print(f"    {line}")
            print("    " + "-" * 72)

if new_changes:
    print("\n✨ NEW Content:")
    for change in new_changes:
        print(f"\n  Page {change.page_number}:")
        print(f"    Checksum: {change.new_checksum[:16]}...")
        print(f"    Content:  {change.new_content[:60]}...")

if deleted_changes:
    print("\n🗑️ DELETED Content:")
    for change in deleted_changes:
        print(f"\n  Page {change.page_number}:")
        print(f"    Checksum: {change.old_checksum[:16]}...")
        print(f"    Content:  {change.old_content[:60]}...")

if unchanged_changes:
    print(f"\n✓ UNCHANGED Content: {len(unchanged_changes)} pages (skipping details)")

📊 Change Summary:
  NEW:       1 (requires FAQ generation)
  MODIFIED:  2 (requires FAQ regeneration)
  DELETED:   1 (requires FAQ invalidation)
  UNCHANGED: 1 (no action needed)


🔄 MODIFIED Content (Similarity Matching Detected These):

  Page 1:
    Old checksum: 86248316e8b0753c...
    New checksum: fb38a20a36d536ca...
    Similarity:   0.985 (>= 0.8 threshold)
    Old content:  # Time Off Policy

Employees are entitled to 10 sick days pe...
    New content:  # Time Off Policy

Employees are entitled to 12 sick days pe...

    📝 LLM-Friendly Diff:
    ------------------------------------------------------------------------
    Found 1 change(s):
    
    CHANGE #1:
    BEFORE: "# Time Off Policy
    
    Employees are entitled to 10 sick days per year."
    AFTER:  "# Time Off Policy
    
    Employees are entitled to 12 sick days per year."
    WHAT CHANGED:
      "10" → "12"
    ------------------------------------------------------------------------

  Page 3:
    Old checksum: 

from granular_impact.similarity.hybrid import HybridSimilarityCalculator

# Initialize similarity calculator (same as used in detector)
similarity_calc = HybridSimilarityCalculator.for_modification_detection()

# =============================================================================
# COMPREHENSIVE TEST SUITE - Realistic Policy Documents
# =============================================================================

test_cases = [
    # -------------------------------------------------------------------------
    # SCENARIO 1: Single Numeric Change (Small)
    # -------------------------------------------------------------------------
    (
        """Time Off Policy - Sick Leave
        
        Full-time employees are entitled to sick leave benefits.
        Each employee receives 10 sick days per calendar year.
        Unused sick days may be carried over to the next year.
        A maximum of 15 days can be accumulated.
        Sick leave requires manager approval for absences over 3 consecutive days.
        Documentation from a healthcare provider may be required.
        Sick leave is prorated for part-time employees.
        Contact HR at hr@company.com for questions.""",
        
        """Time Off Policy - Sick Leave
        
        Full-time employees are entitled to sick leave benefits.
        Each employee receives 12 sick days per calendar year.
        Unused sick days may be carried over to the next year.
        A maximum of 15 days can be accumulated.
        Sick leave requires manager approval for absences over 3 consecutive days.
        Documentation from a healthcare provider may be required.
        Sick leave is prorated for part-time employees.
        Contact HR at hr@company.com for questions.""",
        
        "Single numeric change: 10 → 12 sick days"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 2: Multiple Numeric Changes
    # -------------------------------------------------------------------------
    (
        """Remote Work Policy
        
        Employees may work remotely up to 2 days per week.
        Remote work requires manager approval.
        All remote employees must be available during core hours: 10 AM to 3 PM EST.
        Remote work equipment allowance is $500 per year.
        Internet reimbursement is available up to $50 per month.
        Employees must maintain a dedicated workspace at home.
        VPN access is mandatory for all remote connections.
        Response time for emails should be within 4 hours during business hours.""",
        
        """Remote Work Policy
        
        Employees may work remotely up to 3 days per week.
        Remote work requires manager approval.
        All remote employees must be available during core hours: 9 AM to 4 PM EST.
        Remote work equipment allowance is $750 per year.
        Internet reimbursement is available up to $75 per month.
        Employees must maintain a dedicated workspace at home.
        VPN access is mandatory for all remote connections.
        Response time for emails should be within 2 hours during business hours.""",
        
        "Multiple numeric changes: days (2→3), hours (10AM→9AM, 3PM→4PM), allowance ($500→$750), reimbursement ($50→$75), response time (4h→2h)"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 3: Date Changes
    # -------------------------------------------------------------------------
    (
        """Annual Performance Review Policy
        
        All employees must complete annual performance reviews.
        Review period: January 1, 2024 to December 31, 2024.
        Mid-year check-ins are scheduled for July 15, 2024.
        Final reviews must be submitted by January 31, 2025.
        Self-assessments are due by January 10, 2025.
        Manager reviews are due by January 20, 2025.
        HR will schedule calibration sessions in early February 2025.
        Performance ratings determine annual bonus eligibility.
        Questions should be directed to performance@company.com.""",
        
        """Annual Performance Review Policy
        
        All employees must complete annual performance reviews.
        Review period: January 1, 2025 to December 31, 2025.
        Mid-year check-ins are scheduled for July 15, 2025.
        Final reviews must be submitted by January 31, 2026.
        Self-assessments are due by January 10, 2026.
        Manager reviews are due by January 20, 2026.
        HR will schedule calibration sessions in early February 2026.
        Performance ratings determine annual bonus eligibility.
        Questions should be directed to performance@company.com.""",
        
        "Date changes: 2024 → 2025, 2025 → 2026 (year rollover)"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 4: Small Text Change (Single Word)
    # -------------------------------------------------------------------------
    (
        """Health Insurance Enrollment Policy
        
        All full-time employees are eligible for health insurance benefits.
        Enrollment period opens November 1st each year.
        Coverage becomes effective on January 1st of the following year.
        Employees may choose from three plan options: Basic, Standard, or Premium.
        Dependents may be added to any plan for additional cost.
        Pre-existing conditions are covered without waiting periods.
        Claims should be submitted within 90 days of service.
        Contact benefits@company.com for enrollment assistance.""",
        
        """Health Insurance Enrollment Policy
        
        All full-time employees are eligible for health insurance benefits.
        Enrollment period opens November 1st each year.
        Coverage becomes effective on January 1st of the following year.
        Employees may choose from four plan options: Basic, Standard, Premium, or Executive.
        Dependents may be added to any plan for additional cost.
        Pre-existing conditions are covered without waiting periods.
        Claims should be submitted within 90 days of service.
        Contact benefits@company.com for enrollment assistance.""",
        
        "Small text change: 'three plan options' → 'four plan options' + added plan name"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 5: Large Structural Change (Policy Rewrite)
    # -------------------------------------------------------------------------
    (
        """Vacation Policy - Old Version
        
        Employees accrue vacation time based on tenure.
        0-2 years: 10 days per year.
        3-5 years: 15 days per year.
        6+ years: 20 days per year.
        Vacation must be requested at least 2 weeks in advance.
        Maximum carryover is 5 days per year.
        Unused vacation is forfeited at year end.
        Managers may deny requests during peak business periods.""",
        
        """Unlimited Vacation Policy - New Version
        
        We are transitioning to an unlimited vacation policy.
        Employees may take vacation time as needed with manager approval.
        There is no accrual or carryover tracking.
        We trust employees to manage their time responsibly.
        Minimum recommended vacation is 3 weeks per year.
        All vacation must be coordinated with your team.
        Peak business periods may require restricted time off.
        Policy effective March 1, 2025.""",
        
        "Major policy rewrite: Accrual-based → Unlimited vacation"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 6: Identical Content (No Change)
    # -------------------------------------------------------------------------
    (
        """Code of Conduct
        
        All employees must adhere to the company code of conduct.
        Treat all colleagues with respect and professionalism.
        Discrimination and harassment of any kind will not be tolerated.
        Conflicts of interest must be disclosed to your manager.
        Company resources should be used for business purposes only.
        Confidential information must be protected at all times.
        Violations may result in disciplinary action up to and including termination.
        Report concerns to ethics@company.com or call the hotline at 1-800-555-0123.""",
        
        """Code of Conduct
        
        All employees must adhere to the company code of conduct.
        Treat all colleagues with respect and professionalism.
        Discrimination and harassment of any kind will not be tolerated.
        Conflicts of interest must be disclosed to your manager.
        Company resources should be used for business purposes only.
        Confidential information must be protected at all times.
        Violations may result in disciplinary action up to and including termination.
        Report concerns to ethics@company.com or call the hotline at 1-800-555-0123.""",
        
        "Identical content: No changes"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 7: Mixed Changes (Numbers + Dates + Text)
    # -------------------------------------------------------------------------
    (
        """Parental Leave Policy
        
        New parents are eligible for paid parental leave.
        Birth parents receive 12 weeks of fully paid leave.
        Non-birth parents receive 6 weeks of fully paid leave.
        Leave must be taken within 12 months of birth or adoption.
        Employees must notify HR at least 30 days in advance when possible.
        Benefits continue during leave period.
        Return to work date must be confirmed 2 weeks prior.
        Policy effective January 1, 2024.
        Questions: leave@company.com or call 555-0100.""",
        
        """Enhanced Parental Leave Policy
        
        New parents are eligible for paid parental leave.
        Birth parents receive 16 weeks of fully paid leave.
        Non-birth parents receive 8 weeks of fully paid leave.
        Leave must be taken within 18 months of birth or adoption.
        Employees must notify HR at least 45 days in advance when possible.
        Benefits and stock vesting continue during leave period.
        Return to work date must be confirmed 3 weeks prior.
        Policy effective April 1, 2025.
        Questions: leave@company.com or call 555-0100.""",
        
        "Mixed changes: Numbers (12→16, 6→8, 12→18, 30→45, 2→3), Date (Jan 1, 2024 → Apr 1, 2025), Text ('Enhanced', 'stock vesting')"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 8: Subtle Wording Change (Same Meaning)
    # -------------------------------------------------------------------------
    (
        """Email Policy
        
        Company email should be used for business purposes.
        Personal use should be minimal and not interfere with work.
        Employees are responsible for the content of their emails.
        Email is subject to monitoring and review by the company.
        Confidential information should be marked as such.
        Use encryption for sensitive data.
        Attachments should be scanned for viruses.
        Retain emails according to the company retention policy.""",
        
        """Email Policy
        
        Company email must be used for business purposes.
        Personal use should be limited and not interfere with work.
        Employees are responsible for the content of their messages.
        Email may be monitored and reviewed by the company.
        Confidential information should be clearly marked.
        Encryption is required for sensitive data.
        All attachments must be scanned for viruses.
        Emails should be retained according to company retention policy.""",
        
        "Subtle wording: 'should' → 'must', 'minimal' → 'limited', 'emails' → 'messages', 'is subject to' → 'may be', etc."
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 9: Addition of New Content (Expansion)
    # -------------------------------------------------------------------------
    (
        """Travel Policy - Basic
        
        Business travel must be pre-approved by your manager.
        Use the company travel portal for all bookings.
        Economy class is standard for flights under 5 hours.
        Hotel accommodations should not exceed $200 per night.
        Meals are reimbursed up to $50 per day.
        Submit expense reports within 30 days of travel.
        Receipts are required for all expenses over $25.""",
        
        """Travel Policy - Comprehensive
        
        Business travel must be pre-approved by your manager.
        Use the company travel portal for all bookings.
        Economy class is standard for flights under 5 hours.
        Business class is permitted for international flights over 8 hours.
        Hotel accommodations should not exceed $200 per night in domestic locations.
        International hotel rate is $300 per night maximum.
        Meals are reimbursed up to $50 per day domestic, $75 international.
        Ground transportation includes taxis, rideshare, and public transit.
        Rental cars require advance approval for trips over 3 days.
        Submit expense reports within 30 days of travel.
        Receipts are required for all expenses over $25.
        Travel insurance is provided for international trips over 7 days.""",
        
        "Content expansion: Added international travel rules, ground transportation, rental cars, insurance"
    ),
    
    # -------------------------------------------------------------------------
    # SCENARIO 10: Completely Different Content
    # -------------------------------------------------------------------------
    (
        """Parking Policy
        
        Employees may park in designated company parking areas.
        Parking permits are required and available from facilities.
        Visitor parking is available in the front lot.
        Do not park in reserved or handicap spaces without authorization.
        Overnight parking requires security approval.
        The company is not responsible for damage or theft.
        Violations may result in towing at owner's expense.
        Contact facilities@company.com for parking issues.""",
        
        """Equipment Return Policy
        
        All company equipment must be returned upon termination.
        This includes laptops, monitors, keyboards, mice, and phones.
        Equipment should be returned to IT within 3 business days.
        Data will be wiped from all returned devices.
        Failure to return equipment may result in final paycheck deduction.
        Damaged equipment may be charged to the employee.
        Remote employees should ship equipment to IT using prepaid labels.
        Contact it@company.com for return instructions.""",
        
        "Completely different: Parking → Equipment return (different topic entirely)"
    ),
]

# =============================================================================
# RUN COMPREHENSIVE SIMILARITY ANALYSIS
# =============================================================================

print("=" * 100)
print("COMPREHENSIVE SIMILARITY ANALYSIS - Policy Document Changes")
print("=" * 100)
print()

results = []

for i, (text1, text2, description) in enumerate(test_cases, 1):
    print(f"\n{'='*100}")
    print(f"TEST CASE {i}: {description}")
    print('='*100)
    
    result = similarity_calc.compute_similarity(text1, text2)
    score = result.score
    
    # Decision logic (same as detector)
    if score >= 0.8:
        decision = "✅ MODIFIED (will regenerate FAQ)"
    else:
        decision = "❌ NEW/DELETED (completely different content)"
    
    # Safely access individual scores (handle early exit case)
    individual_scores = result.metadata.get('individual_scores', {})
    jaccard_score = individual_scores.get('jaccard', 'N/A')
    difflib_score = individual_scores.get('difflib', 'N/A')
    bm25_score = individual_scores.get('bm25', 'N/A')
    
    # Format scores
    jaccard_str = f"{jaccard_score:.3f}" if isinstance(jaccard_score, float) else jaccard_score
    difflib_str = f"{difflib_score:.3f}" if isinstance(difflib_score, float) else difflib_score
    bm25_str = f"{bm25_score:.3f}" if isinstance(bm25_score, float) else bm25_score
    
    print(f"\n📊 SIMILARITY SCORE: {score:.3f}")
    print(f"🎯 DECISION: {decision}")
    print(f"\n🔍 Algorithm Breakdown:")
    print(f"   • Jaccard:  {jaccard_str}  (20% weight) - Token overlap")
    print(f"   • Difflib:  {difflib_str}  (50% weight) - Character-level matching")
    print(f"   • BM25:     {bm25_str}  (30% weight) - Ranking score")
    
    # Show if early exit was triggered
    if result.metadata.get('early_exit', False):
        print(f"\n⚡ Early exit triggered (Jaccard < {result.metadata['early_exit_threshold']})")
        print(f"   Skipped algorithms: {', '.join(result.metadata.get('algorithms_skipped', []))}")
    
    # Show sample text differences
    lines1 = text1.strip().split('\n')
    lines2 = text2.strip().split('\n')
    print(f"\n📝 Content Stats:")
    print(f"   • Original:  {len(text1)} chars, {len(lines1)} lines, {len(text1.split())} words")
    print(f"   • Modified:  {len(text2)} chars, {len(lines2)} lines, {len(text2.split())} words")
    print(f"   • Change:    {len(text2) - len(text1):+d} chars, {len(lines2) - len(lines1):+d} lines")
    
    # Store results for summary
    results.append({
        'test_case': i,
        'description': description,
        'score': score,
        'decision': decision,
        'jaccard': jaccard_score,
        'difflib': difflib_score,
        'bm25': bm25_score,
        'early_exit': result.metadata.get('early_exit', False)
    })

# =============================================================================
# SUMMARY TABLE
# =============================================================================

print(f"\n\n{'='*100}")
print("SUMMARY TABLE")
print('='*100)
print()
print(f"{'#':<3} {'Score':<7} {'Decision':<12} {'Jaccard':<8} {'Difflib':<8} {'BM25':<8} {'Description':<50}")
print('-'*100)

for r in results:
    jaccard_str = f"{r['jaccard']:.3f}" if isinstance(r['jaccard'], float) else "N/A"
    difflib_str = f"{r['difflib']:.3f}" if isinstance(r['difflib'], float) else "N/A"
    bm25_str = f"{r['bm25']:.3f}" if isinstance(r['bm25'], float) else "N/A"
    decision_short = "MODIFIED" if "MODIFIED" in r['decision'] else "NEW/DEL"
    
    print(f"{r['test_case']:<3} {r['score']:<7.3f} {decision_short:<12} {jaccard_str:<8} {difflib_str:<8} {bm25_str:<8} {r['description'][:48]:<50}")

print()
print("=" * 100)
print("✅ Comprehensive analysis complete!")
print("=" * 100)

In [9]:
from granular_impact.similarity.hybrid import HybridSimilarityCalculator

# Initialize similarity calculator (same as used in detector)
similarity_calc = HybridSimilarityCalculator.for_modification_detection()

# Test different types of changes
test_cases = [
    (
        "Employees are entitled to 10 sick days per year.",
        "Employees are entitled to 12 sick days per year.",
        "Minor numeric change"
    ),
    (
        "Employees may work remotely up to 2 days per week.",
        "Employees may work remotely up to 3 days per week.",
        "Minor numeric change"
    ),
    (
        "Full-time employees receive health insurance coverage.",
        "Full-time employees receive health insurance coverage.",
        "Identical content"
    ),
    (
        "Employees are entitled to 10 sick days per year.",
        "Employees receive 12 weeks of paid parental leave.",
        "Completely different content"
    ),
    (
        "The policy effective date is January 1, 2024.",
        "The policy effective date is February 15, 2024.",
        "Date change"
    ),
]

print("🧪 Similarity Matching Tests:")
print("=" * 80)

for text1, text2, description in test_cases:
    result = similarity_calc.compute_similarity(text1, text2)
    score = result.score
    
    # Decision logic (same as detector)
    if score >= 0.8:
        decision = "MODIFIED (will regenerate FAQ)"
    else:
        decision = "NEW/DELETED (completely different)"
    
    print(f"\n{description}:")
    print(f"  Text 1: {text1}")
    print(f"  Text 2: {text2}")
    print(f"  Similarity: {score:.3f}")
    print(f"  Decision: {decision}")
    
    # Safely access individual scores (handle early exit case)
    individual_scores = result.metadata.get('individual_scores', {})
    jaccard_score = individual_scores.get('jaccard', 'N/A')
    difflib_score = individual_scores.get('difflib', 'N/A')
    bm25_score = individual_scores.get('bm25', 'N/A')
    
    # Format scores (handle N/A for skipped algorithms)
    jaccard_str = f"{jaccard_score:.3f}" if isinstance(jaccard_score, float) else jaccard_score
    difflib_str = f"{difflib_score:.3f}" if isinstance(difflib_score, float) else difflib_score
    bm25_str = f"{bm25_score:.3f}" if isinstance(bm25_score, float) else bm25_score
    
    print(f"  Breakdown: Jaccard={jaccard_str}, Difflib={difflib_str}, BM25={bm25_str}")
    
    # Show if early exit was triggered
    if result.metadata.get('early_exit', False):
        print(f"  ⚡ Early exit triggered (Jaccard < {result.metadata['early_exit_threshold']})")

🧪 Similarity Matching Tests:

Minor numeric change:
  Text 1: Employees are entitled to 10 sick days per year.
  Text 2: Employees are entitled to 12 sick days per year.
  Similarity: 0.979
  Decision: MODIFIED (will regenerate FAQ)
  Breakdown: Jaccard=N/A, Difflib=0.979, BM25=N/A

Minor numeric change:
  Text 1: Employees may work remotely up to 2 days per week.
  Text 2: Employees may work remotely up to 3 days per week.
  Similarity: 0.980
  Decision: MODIFIED (will regenerate FAQ)
  Breakdown: Jaccard=N/A, Difflib=0.980, BM25=N/A

Identical content:
  Text 1: Full-time employees receive health insurance coverage.
  Text 2: Full-time employees receive health insurance coverage.
  Similarity: 1.000
  Decision: MODIFIED (will regenerate FAQ)
  Breakdown: Jaccard=N/A, Difflib=1.000, BM25=N/A

Completely different content:
  Text 1: Employees are entitled to 10 sick days per year.
  Text 2: Employees receive 12 weeks of paid parental leave.
  Similarity: 0.408
  Decision: NEW/DELETED (

---

## Part 5: Database Integration (SQLite Example)

In production, you'll:
1. Retrieve previous checksums from database
2. Store change detection results
3. Query FAQs affected by changes

Let's demonstrate with a local SQLite database.

In [ ]:
import sqlite3

# Create an in-memory SQLite database (for demonstration)
conn = sqlite3.connect(":memory:")

# Create tables (simplified V8.2 schema)
conn.execute("""
    CREATE TABLE content_checksums (
        content_checksum TEXT PRIMARY KEY,
        content_text TEXT,
        file_name TEXT,
        page_number INTEGER,
        section_name TEXT,
        status TEXT DEFAULT 'active',
        created_at TEXT,
        markdown_file_path TEXT
    )
""")

conn.execute("""
    CREATE TABLE content_change_log (
        change_id INTEGER PRIMARY KEY AUTOINCREMENT,
        content_checksum TEXT,
        previous_checksum TEXT,
        file_name TEXT,
        page_number INTEGER,
        section_name TEXT,
        requires_faq_regeneration BOOLEAN,
        change_type TEXT,
        similarity_score REAL,
        similarity_method TEXT,
        total_faqs_at_risk INTEGER DEFAULT 0,
        affected_question_count INTEGER DEFAULT 0,
        affected_answer_count INTEGER DEFAULT 0,
        detection_run_id TEXT,
        detection_timestamp TEXT
    )
""")

# Insert previous checksums into database
for checksum, data in previous_checksums_data.items():
    conn.execute(
        "INSERT INTO content_checksums VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
        (
            checksum,
            data["content_text"],
            data["file_name"],
            data["page_number"],
            None,  # section_name
            "active",
            datetime.now().isoformat(),
            None,  # markdown_file_path
        )
    )

conn.commit()
print("✅ Created in-memory database and inserted previous checksums")

# Verify
cursor = conn.execute("SELECT COUNT(*) FROM content_checksums WHERE status = 'active'")
count = cursor.fetchone()[0]
print(f"   Database contains {count} active content checksums")

### Retrieve Previous Checksums from Database

In [ ]:
# Use ChangeDetectionQueries to retrieve previous checksums
retrieved_checksums = ChangeDetectionQueries.get_previous_checksums_for_file(
    conn=conn,
    file_name="hr_handbook.pdf",
    use_spark=False  # Using SQLite, not Spark
)

print(f"📊 Retrieved {len(retrieved_checksums)} checksums from database")
print(f"\nSample entry:")
sample_key = list(retrieved_checksums.keys())[0]
print(f"Checksum: {sample_key[:16]}...")
print(f"Data: {retrieved_checksums[sample_key]}")

### Store Change Detection Results

In [ ]:
# Store the changes we detected earlier
inserted_count = ChangeDetectionQueries.store_change_detection_results(
    conn=conn,
    changes=changes,
    detection_run_id=detection_run_id,
    use_spark=False
)

print(f"✅ Stored {inserted_count} change records")

# Query the results
cursor = conn.execute("""
    SELECT 
        change_type,
        COUNT(*) as count,
        AVG(similarity_score) as avg_similarity
    FROM content_change_log
    GROUP BY change_type
    ORDER BY change_type
""")

print("\n📊 Changes stored in database:")
for row in cursor.fetchall():
    change_type, count, avg_sim = row
    print(f"  {change_type}: {count} changes (avg similarity: {avg_sim or 0:.3f})")

# Show modified content details
cursor = conn.execute("""
    SELECT 
        page_number,
        change_type,
        similarity_score,
        requires_faq_regeneration
    FROM content_change_log
    WHERE requires_faq_regeneration = 1
    ORDER BY page_number
""")

print("\n⚠️ Changes requiring FAQ regeneration:")
for row in cursor.fetchall():
    page, change_type, similarity, requires_regen = row
    print(f"  Page {page}: {change_type} (similarity: {similarity or 0:.3f})")

---

## Part 6: Production Workflow

Here's how you'd use this in a real Databricks notebook for FAQ regeneration:

In [ ]:
# Example production workflow (pseudo-code for Databricks)

def production_workflow_example():
    """
    Production workflow for FAQ regeneration detection.
    
    This would run in a Databricks notebook.
    """
    
    # Step 1: Extract checksums from NEW content (e.g., newly extracted PDFs)
    print("Step 1: Extracting checksums from new content...")
    # file_extractor = ChecksumFileExtractor()
    # current_checksums = file_extractor.extract_from_markdown_files(
    #     markdown_dir="/Volumes/onedata_us_east_1_shared_dit/faqservice/content/hr_handbook/2025_01_15/"
    # )
    
    # Step 2: Retrieve previous checksums from database/Delta table
    print("Step 2: Retrieving previous checksums from database...")
    # from pyspark.sql import SparkSession
    # spark = SparkSession.builder.getOrCreate()
    # previous_checksums = ChangeDetectionQueries.get_previous_checksums_for_file(
    #     conn=spark,
    #     file_name="hr_handbook.pdf",
    #     use_spark=True
    # )
    
    # Step 3: Detect changes
    print("Step 3: Detecting content changes...")
    # detector = ContentChangeDetector(similarity_threshold=0.8)
    # changes = detector.detect_changes(
    #     file_name="hr_handbook.pdf",
    #     current_checksums_data=current_checksums,
    #     previous_checksums_data=previous_checksums,
    #     detection_run_id=f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    # )
    
    # Step 4: Store results in Delta table
    print("Step 4: Storing change detection results...")
    # ChangeDetectionQueries.store_change_detection_results(
    #     conn=spark,
    #     changes=changes,
    #     detection_run_id=detection_run_id,
    #     use_spark=True
    # )
    
    # Step 5: Identify FAQs requiring regeneration
    print("Step 5: Identifying affected FAQs...")
    # for change in changes:
    #     if change.change_type in [ChangeType.MODIFIED_CONTENT, ChangeType.NEW_CONTENT]:
    #         affected_faqs = ChangeDetectionQueries.get_faqs_for_checksum(
    #             conn=spark,
    #             checksum=change.old_checksum or change.new_checksum,
    #             use_spark=True
    #         )
    #         # Process affected FAQs (invalidate or regenerate)
    
    # Step 6: Trigger FAQ regeneration pipeline
    print("Step 6: Triggering FAQ regeneration for affected content...")
    # from faq_services.utils.path_config import PathConfig
    # config = PathConfig(sor="analytics_assist", tag="questions_and_answers_generation")
    # # Run Hydra pipeline for regeneration...
    
    print("✅ Workflow complete!")

# Display the workflow
print("📋 Production Workflow Steps:")
print("=" * 80)
production_workflow_example()

---

## Part 7: Key Takeaways and Best Practices

### What We Learned

1. **Checksums are content identities**
   - SHA-256 produces unique 64-character fingerprints
   - Even tiny changes ("10"→"12") produce different checksums
   - Location (page number, file name) is just metadata

2. **Change detection is set-based**
   - Current checksums - Previous checksums = New content
   - Previous checksums - Current checksums = Deleted content
   - Intersection = Unchanged content

3. **Similarity matching catches modifications**
   - Without it, "10 days"→"12 days" looks like DELETE + NEW
   - With it, we correctly identify MODIFIED content
   - Threshold of 0.8 (80%) works well for policy documents

4. **Hybrid similarity is powerful**
   - Jaccard: Token-level overlap (fast)
   - Difflib: Character-level accuracy (catches numeric changes)
   - BM25: Ranking-based similarity
   - Combined: Robust to different types of changes

### Best Practices

1. **Always scope checksums to a file**
   - Don't compare global checksum sets
   - Query: `WHERE file_name = 'hr_handbook.pdf'`

2. **Use temporal filtering**
   - Get previous checksums created BEFORE current detection run
   - Avoids comparing with checksums from the same run

3. **Configure similarity threshold carefully**
   - 0.8 = Good for policy documents (catches "10"→"12")
   - 0.9 = More strict (only very similar content = MODIFIED)
   - 0.7 = More lenient (more content = MODIFIED)

4. **Store change detection results**
   - Enables auditing ("Why did we regenerate this FAQ?")
   - Supports incremental processing
   - Tracks similarity scores for analysis

5. **Handle edge cases**
   - Content splits (1 page → 2 pages): Detector logs warnings
   - Content merges (2 pages → 1 page): All new checksums flagged
   - Empty content: File extractor logs warnings

### Next Steps

After change detection, you need:
1. **Impact Analysis**: Which FAQs are affected?
2. **Selective Invalidation**: Mark specific Q/A as invalid
3. **FAQ Regeneration**: Generate new FAQs for changed content

These are covered in other modules:
- `granular_impact/impact/` - Impact scoring and decision
- `granular_impact/workflow/` - Orchestration and invalidation
- `granular_impact/diff/` - Granular diff analysis

---

## Cleanup

In [ ]:
# Close database connection
conn.close()

# Optionally: Clean up sample files
import shutil
if sample_dir.exists():
    shutil.rmtree(sample_dir)
    print(f"✅ Cleaned up sample directory: {sample_dir}")

print("\n🎉 Notebook complete! You now understand how to use the detection module.")